In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [17]:
%%writefile rx_mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t',1)
        count=int(count)
        print "%d\t%s" % (count,key)
    except ValueError as e:
        continue

Overwriting rx_mapper.py


In [18]:
%%writefile rx_reducer.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        count,key = line.strip().split('\t',1)
        count=int(count)
        print "%s\t%d" % (key, count)
    except ValueError as e:
        continue

Overwriting rx_reducer.py


In [6]:
!hdfs dfs -mkdir /data/wiki/gaio_files

In [7]:
! hdfs dfs -ls /data/wiki/gaio_files

In [8]:
!hdfs dfs -ls /data/wiki/en_articles_part/articles-part/

-rwxrwxrwx   1 jovyan supergroup   76861985 2017-10-17 13:15 /data/wiki/en_articles_part/articles-part


In [9]:
!hdfs dfs -cat /data/wiki/en_articles_part/articles-part | head > a.txt

cat: Unable to write to output stream.


In [10]:
%ls

20180514-wordcount-reverse-WORK.ipynb  README.md      supervisord.log
a.txt                                  reducer.py     supervisord.pid
DemoNotebook.ipynb                     rx_mapper.py
mapper.py                              rx_reducer.py


In [11]:
!hdfs dfs -copyFromLocal a.txt /data/wiki/gaio_files

In [12]:
! hdfs dfs -ls /data/wiki/gaio_files

Found 1 items
-rw-r--r--   1 jovyan supergroup     424525 2018-05-14 18:49 /data/wiki/gaio_files/a.txt


In [27]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/gaio_files \
    -output ${OUT_DIR} > /dev/null

#hdfs dfs -cat ${OUT_DIR}/part-00000 | head

OUT_DIR_REV="wordcount_rev_"$(date +"%s%6N")
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_REV} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming rev" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options=-nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files rx_mapper.py,rx_reducer.py \
    -mapper "python rx_mapper.py" \
    -reducer "python rx_reducer.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_REV} > /dev/null

hdfs dfs -cat ${OUT_DIR_REV}/part-00000 | head -7 | tail -1

is	584


rm: `wordcount_result_1526324470446091': No such file or directory
18/05/14 19:01:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/14 19:01:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/14 19:01:14 INFO mapred.FileInputFormat: Total input files to process : 1
18/05/14 19:01:14 INFO mapreduce.JobSubmitter: number of splits:2
18/05/14 19:01:14 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1526287196433_0011
18/05/14 19:01:14 INFO impl.YarnClientImpl: Submitted application application_1526287196433_0011
18/05/14 19:01:14 INFO mapreduce.Job: The url to track the job: http://55e59bd90628:8088/proxy/application_1526287196433_0011/
18/05/14 19:01:14 INFO mapreduce.Job: Running job: job_1526287196433_0011
18/05/14 19:01:20 INFO mapreduce.Job: Job job_1526287196433_0011 running in uber mode : false
18/05/14 19:01:20 INFO mapreduce.Job:  map 0% reduce 0%
18/05/14 19:01:24 INFO mapreduce.Job:  map 50% reduce 0%
18/05/14 19:0

In [20]:
!hdfs dfs -ls wordcount_rev_1526324102803277

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2018-05-14 18:55 wordcount_rev_1526324102803277/_SUCCESS
-rw-r--r--   1 jovyan supergroup     125310 2018-05-14 18:55 wordcount_rev_1526324102803277/part-00000


In [23]:
!hdfs dfs -cat wordcount_rev_1526324102803277/part-00000 | head -7 | tail -1

is	584
cat: Unable to write to output stream.


In [15]:
%ls

20180514-wordcount-reverse-WORK.ipynb  README.md      supervisord.log
a.txt                                  reducer.py     supervisord.pid
DemoNotebook.ipynb                     rx_mapper.py
mapper.py                              rx_reducer.py
